# Neural Machine Translation


Write down some summary on your experiments and illustrate it with convergence plots/metrics and your thoughts. Just like you would approach a real problem.

Compare results from full and prunned models.

! pip  install subword-nmt
! wget -nc https://raw.githubusercontent.com/ml-mipt/ml-mipt/advanced/homeworks/Lab1_NLP/data.txt -P ./data

# Thanks to YSDA NLP course team for the data
# (who thanks tilda and deephack teams for the data in their turn)

In [1]:
from MyTranslator import BaseDataModule, MyTranslator
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning import loggers as pl_loggers

## Main part
__Here comes the preprocessing. Try to use BPE or more complex preprocessing ;)__


In [2]:
SEED = 42
DEVICE = "cuda"
BATCH_SIZE = 128

data_module = BaseDataModule(
    batch_size=BATCH_SIZE,
    device = DEVICE,
    data_path="./data/data.txt",
    seed=SEED
)

data_module.prepare_data()

In [3]:
LEARNING_RATE = 2e-5
N_EPOCHS = 60
CLIP = 1

params = {
    "n_layers": 3,
    "n_heads": 8,
    "max_length": 100,
    "enc_hid_dim": 512,
    "enc_pf_dim": 512,
    "enc_dropout": 0.3,
    "dec_hid_dim": 512,
    "dec_pf_dim": 512,
    "dec_dropout": 0.3,
    "src_field": data_module.src_field,
    "trg_field": data_module.trg_field, 
    "lr": LEARNING_RATE,
    "device": DEVICE,
}

In [ ]:
model = MyTranslator(**params)
model = model.to(DEVICE)
model.hparams.lr = LEARNING_RATE
tb_logger = pl_loggers.TensorBoardLogger('./logs/')
early_stop_callback = EarlyStopping(
   monitor='avg_val_loss',
   min_delta=0.01,
   patience=2,
   verbose=False,
   mode='mean'
)
trainer = Trainer(
    max_epochs=N_EPOCHS,
    gradient_clip_val=0.5,
    progress_bar_refresh_rate=1,
    callbacks=[early_stop_callback], 
    logger=tb_logger,
    log_every_n_steps=20
    )
data_module.setup('fit')
trainer.fit(model, data_module)

In [6]:
#torch.save(model, "models/myTransformer2")
trainer.save_checkpoint("models/myTransformer.ckpt")
#new_model = MyModel.load_from_checkpoint(checkpoint_path="models/myTransformer2.ckpt")

__Let's take a look at our network quality__:

In [ ]:
trainer.test(model)

Baseline solution BLEU score is quite low. 
The checkpoints are:

* __18__ - baseline

* __20__ - good score

* __25__ - excellent score